Task 1: Third-Order Letter Approximation Model

In [13]:
# Imports.

# Selecting random items from lists.
import random

# Efficient data structures.
import collections

Step 1: A method that will read in the text from one of the books

In [14]:
#method to read in a book
def readBook(book):
    with open('books/'+book, 'r', encoding='utf-8') as file:#open the file
        english = file.read()#store the read in file in a variable

    return english#return the text

In [15]:
#Clean the text by putting it to lower case and only keeping ascii chars
def cleanBook(book):
    #change everything to lower case
    english = book.lower();
    # The characters to keep.
    keep = 'abcdefghijklmnopqrstuvwxyz .'
    # Remove unwanted characters.
    cleaned = ''.join(c for c in english if c in keep)

    return cleaned


In [16]:
text = readBook('voyaging.txt')

print(text)

The Project Gutenberg eBook of David goes voyaging
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: David goes voyaging

Author: David Binney Putnam

Illustrator: Isabel Cooper
        Don Dickerman
        Dwight Franklin

Release date: October 10, 2024 [eBook #74551]

Language: English

Original publication: United States: G. P. Putnam's Sons

Credits: Jeroen Hellingman and the Online Distributed Proofreading Team at https://www.pgdp.net/ for Project Gutenberg (This file was produced from images generously made available by The Internet Archive/American Libraries.)


*** START OF THE 